In [5]:
import pandas as pd
import nltk

import sys
sys.path.append('/Users/jessicahassibi/PycharmProjects/bachelor-thesis/src/data') # replace with your absolute path
# import modules from another folder
import WikiScraper
import data_analysis

In [13]:
# languages in which the articles shall be crawled
languages = ['de', 'en', 'ar', 'fr', 'es', 'it']

# new instance of WikiScraper
wiki_scraper = WikiScraper.WikiScraper()

# Scrape list of persecuted composers
persecuted_composers_page = "Liste der vom NS-Regime oder seinen Verbündeten verfolgten Komponisten"
wiki_scraper.scrape_pages_multiple_languages(persecuted_composers_page, languages)

# Scrape list of supported composers
supported_composers_page = "Gottbegnadeten-Liste"
wiki_scraper.scrape_pages_multiple_languages(supported_composers_page, languages)

New WikiScraper initialized.

Texts have already been scraped to: ../data/persecuted_composers/de_texts_composers_persecuted.json
Texts have already been scraped to: ../data/persecuted_composers/en_texts_composers_persecuted.json
Texts have already been scraped to: ../data/persecuted_composers/ar_texts_composers_persecuted.json
Texts have already been scraped to: ../data/persecuted_composers/fr_texts_composers_persecuted.json
Texts have already been scraped to: ../data/persecuted_composers/es_texts_composers_persecuted.json
Texts have already been scraped to: ../data/persecuted_composers/it_texts_composers_persecuted.json
Texts have already been scraped to: ../data/supported_composers/de_texts_composers_supported.json
Texts have already been scraped to: ../data/supported_composers/en_texts_composers_supported.json
Texts have already been scraped to: ../data/supported_composers/ar_texts_composers_supported.json
Texts have already been scraped to: ../data/supported_composers/fr_texts_com

In [14]:
# english dataframe of supported composers
en_df_supported = data_analysis.get_dataframe_from_path("../data/supported_composers/en_texts_composers_supported.json")
# english dataframe of persecuted^  ^ composers
en_df_persecuted = data_analysis.get_dataframe_from_path("../data/persecuted_composers/en_texts_composers_persecuted.json")

In [25]:
# create english train dataframe using texts and labels
en_df = pd.DataFrame()
en_df["text"] = pd.concat([en_df_supported["text"], en_df_persecuted["text"]], ignore_index=True)
en_df["label"] = pd.concat([en_df_supported["label"], en_df_persecuted["label"]], ignore_index=True)


In [16]:
import os
os.makedirs('../data/dataframes', exist_ok=True)
en_df.to_csv(f'../data/dataframes/en_df.csv')

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

# TODO: make it work for all langs
# TODO: numbers/years  as stop words?
def clean_texts(text, lang="english"):
    """
    Cleans data by applying tokenization, removal of stop words
    """
    tokens = []
    punctuation_marks = [",", ".", ";", ":", "-", '"', "/",'""', "''" "'", "(", ")", "[", "]", "!", "?", "=", "{", "}", "&"]
    for sent in sent_tokenize(text, lang):
        for token in word_tokenize(sent, lang):
            token = token.lower()
            if not is_stop_word(token, lang) and token not in punctuation_marks:
                # additionally lemmatize?!
                tokens.append(token)
    return tokens

def is_stop_word(word, lang):
    sw = stopwords.words(lang)
    sw.append("isbn")
    if word in sw:
        return True
    elif re.match(r"\d+", word): # e.g. "1" or "20" but not a year (1994) as it may contain historical information:
        return True
    else:
        return False


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jessicahassibi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# TODO: geburtsdaten cleanen
# create new dataframe with cleaned text
language = "english"
en_df_cleaned = en_df
en_df_cleaned.insert(1, "cleaned_text", en_df_cleaned["text"].apply(lambda x: clean_texts(x, language)))
en_df_cleaned.drop(labels="text",axis="columns", inplace=True)

In [27]:
# Encoding the label column
en_df_cleaned['label']=en_df_cleaned['label'].map({'supported':1,'persecuted':0})

In [28]:
# save cleaned dataframe
en_df_cleaned.to_csv(f'../data/dataframes/en_df_cleaned.csv')